# Data Exploration

In [1]:
#@title import packages

import gunpowder as gp
import h5py
import io
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import requests
import torch
import waterz
import zarr

from funlib.learn.torch.models import UNet, ConvPass
from gunpowder.torch import Predict
from scipy.ndimage import label, measurements
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import distance_transform_edt
from skimage.segmentation import watershed

%matplotlib inline
logging.basicConfig(level=logging.INFO)

/tmp/ipykernel_3239/2871270234.py:20: DeprecationWarning: Please use `maximum_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import maximum_filter
/tmp/ipykernel_3239/2871270234.py:21: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


In [2]:
file_path = '/data/lsd_nm_experiments/03_salk/salk/3M-APP-SCN/training/data.zarr'

data = zarr.open(file_path, 'r')

raw = data['volumes/raw']
labels = data['volumes/labels/neuron_ids']

print('raw.shape', raw.shape)
print('labels.shape', labels.shape)

# container = zarr.open(name, 'a')

# if sections is None:
#   sections=range(raw.shape[0]-1)

# for index, section in enumerate(sections):

#   print(f'Writing data for section {section}')

#   raw_slice = raw[section]
#   labels_slice = labels[section]

#   if squeeze:
#     raw_slice = np.squeeze(raw_slice)
#     labels_slice = np.squeeze(labels_slice)

#   for ds_name, data in [
#       ('raw', raw_slice),
#       ('labels', labels_slice)]:

#       container[f'{ds_name}/{index}'] = data
#       container[f'{ds_name}/{index}'].attrs['offset'] = offset
#       container[f'{ds_name}/{index}'].attrs['resolution'] = resolution

raw.shape (100, 1000, 1000)
labels.shape (100, 1000, 1000)


In [34]:
def print_zarr_hierarchy(obj, name=''):
    if isinstance(obj, zarr.Group):
        print(f"Group: {name}")
        for key in obj.keys():
            print_zarr_hierarchy(obj[key], name=f"{name}/{key}")
    elif isinstance(obj, zarr.Array):
        print(f"Array: {name}, Shape: {obj.shape}, dtype: {obj.dtype}")
    elif isinstance(obj, zarr.Attributes):
        print(f"Attributes: {name}")
        for attr_name in obj.attrs_keys():
            print(f"Attribute: {attr_name}, Value: {obj.attrs[attr_name]}")

print_zarr_hierarchy(data)

Group: 
Group: /volumes
Group: /volumes/labels
Array: /volumes/labels/labels_mask, Shape: (100, 1000, 1000), dtype: uint8
Array: /volumes/labels/neuron_ids, Shape: (100, 1000, 1000), dtype: uint64
Array: /volumes/raw, Shape: (100, 1000, 1000), dtype: uint8


In [12]:
lab = np.array(labels)
lab.shape


(100, 1000, 1000)

In [15]:
np.sum(lab == 41)

36542225

In [16]:
np.sum(lab == 0)

22938193

In [17]:
np.sum(lab == 1)

11741

## create_masks.py

Custom script for 3M-APP-SCN data

In [35]:
import numpy as np
import zarr

file_path = '/data/lsd_nm_experiments/03_salk/salk/3M-APP-SCN/training/data.zarr'

container = zarr.open(file_path, "a")

labels = container["volumes/labels/neuron_ids"]
offset = labels.attrs["offset"]
resolution = labels.attrs["resolution"]

labels = labels[:]

# true where labels == true, can be used to get network to make predictions in
# background regions (in this case these regions correspond to glia)
unlabelled = (labels > 0).astype(np.uint8)

# label 41 is a big cell body. Let's also create a mask without this label that
# we can use to restrict random locations to mostly neuropil regions
object_mask = unlabelled.copy()
object_mask[labels == 41] = 0

# write data to zarr
for ds_name, data in [("object_mask", object_mask), ("unlabelled", unlabelled)]:
    container[f"volumes/labels/{ds_name}"] = data
    container[f"volumes/labels/{ds_name}"].attrs["offset"] = offset
    container[f"volumes/labels/{ds_name}"].attrs["resolution"] = resolution
